# Section I: Basic Array Operations

* To enter REPL, type `julia` in your terminal (PATH variable in shell should be set correctly).
* To get help in REPL, type `?` to enter `API` mode.
* To install a package, type `]` to enter `Pkg` mode. Type `?<Enter>` if you want some help in `Pkg` mode.
* To run shell command in REPL, type `;` to enter shell mode.
* To input `≈`, type `\approx<TAB>`, see full list of unicode input of julia [here](https://docs.julialang.org/en/v1/manual/unicode-input/).
* `@test` means stop program and throw and exception whenever a clause returns `false`. "function" start with `@` character is called a [macro](https://docs.julialang.org/en/v1/manual/metaprogramming/).

In [2]:
# Julia Matrix Operation is fast. To know how fast it is, you need a benchmark package. Type `]` and `add BenchmarkTools<Enter>` in your REPL.
using BenchmarkTools, Test

cnot = [1 0 0 0; 0 1 0 0; 0 0 0 1; 0 0 1 0]  # a matrix
b = [1, 1im, true, 0.4e2] # a vector
c = [1, 1im, 0.4e2, true] # a vector

# run tests
@test cnot*b ≈ c
@test_throws DimensionMismatch cnot*randn(3)  # expected to raise specific error

# run a benchmark
res1 = @benchmark $cnot*$b  # `$` sign means evaluation first, used in `@benchmark` to avoid taking time to evaluate cnot input account.

BenchmarkTools.Trial: 
  memory estimate:  144 bytes
  allocs estimate:  1
  --------------
  minimum time:     55.177 ns (0.00% GC)
  median time:      59.870 ns (0.00% GC)
  mean time:        73.669 ns (14.85% GC)
  maximum time:     61.859 μs (99.82% GC)
  --------------
  samples:          10000
  evals/sample:     979

# Can it be faster? Yes if the matrix is small and static!

For static array, we can avoid all allocations.

### **Used in**
* Quantum Circuit Simulation

### **Challenge!**
Read the document of package `StaticArrays`
https://github.com/JuliaArrays/StaticArrays.jl
and show it is really fast!

In [3]:
# install `StaticArrays`

using StaticArrays: SMatrix, SVector
# sa = <some static matrix> 
# sb = <some static vector> 

@testset "static arrays" begin
    @test scnot*sb ≈ c
    using Statistics: median
    res2 = @benchmark $scnot*$sb
    @test median(res2).time < median(res1).time / 5  # at least 5 times faster
end

static arrays: Error During Test at In[3]:8
  Test threw exception
  Expression: scnot * sb ≈ c
  UndefVarError: scnot not defined
  Stacktrace:
   [1] macro expansion at ./In[3]:8 [inlined]
   [2] macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Test/src/Test.jl:1083 [inlined]
   [3] top-level scope at ./In[3]:8
static arrays: Error During Test at In[3]:7
  Got exception outside of a @test
  UndefVarError: scnot not defined
  Stacktrace:
   [1] macro expansion at /home/leo/.julia/packages/BenchmarkTools/dtwnm/src/execution.jl:258 [inlined]
   [2] macro expansion at ./In[3]:10 [inlined]
   [3] macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Test/src/Test.jl:1083 [inlined]
   [4] top-level scope at ./In[3]:8
   [5] eval at ./boot.jl:319 [inlined]
   [6] softscope_include_string(::Module, ::String, ::String) at /home/leo/.julia/packages/SoftGlobalScope/ujmiK/src/SoftGlobalScope.jl:206
   [7] execute_request(

TestSetException: Some tests did not pass: 0 passed, 0 failed, 2 errored, 0 broken.

# Linear Algebra

See [docs](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/).

### **Used in**
* Quantum Monte Carlo
* Non-Interacting fermionic systems (include mean field)

### **Challenge!**
Try to figure out how to use get eigenvalues, singular values and qr and pass the test

In [4]:
using LinearAlgebra: eigen, svd, qr, I, det, tr
# hint, here you probabily want to type `?` and `eigen<Enter>` to get help in an REPL.

# sv_cnot = <get the singular value of matrix cnot>
# ev_cnot = <get the eigenvalue of matrix cnot>
# q_cnot = <get the Q matrix from QR decomposition of cnot>
# det_cnot = <get the determinant of matrix cnot>
# tr_cnot = <get the trace of matrix cnot>

@testset "linalg" begin
    @test sv_cnot ≈ ones(4)
    @test ev_cnot ≈ [-1, 1, 1, 1]
    @test q_cnot*q_cnot' ≈ I
    @test det_cnot == -1
    @test tr_cnot == 2
end

linalg: Error During Test at In[4]:11
  Test threw exception
  Expression: sv_cnot ≈ ones(4)
  UndefVarError: sv_cnot not defined
  Stacktrace:
   [1] macro expansion at ./In[4]:11 [inlined]
   [2] macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Test/src/Test.jl:1083 [inlined]
   [3] top-level scope at ./In[4]:11
linalg: Error During Test at In[4]:12
  Test threw exception
  Expression: ev_cnot ≈ [-1, 1, 1, 1]
  UndefVarError: ev_cnot not defined
  Stacktrace:
   [1] macro expansion at ./In[4]:12 [inlined]
   [2] macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Test/src/Test.jl:1083 [inlined]
   [3] top-level scope at ./In[4]:11
linalg: Error During Test at In[4]:13
  Test threw exception
  Expression: q_cnot * q_cnot' ≈ I
  UndefVarError: q_cnot not defined
  Stacktrace:
   [1] macro expansion at ./In[4]:13 [inlined]
   [2] macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdli

TestSetException: Some tests did not pass: 0 passed, 0 failed, 5 errored, 0 broken.

# Large sparse matrix eigensolver

[KrylovKit](https://github.com/Jutho/KrylovKit.jl) is a package for solving large sparse matrix.


### **Used in**
* Exact diagonalization
* Cluster pertubation theory (CPT)
* Numerical Renomalization Group

### **Challenge!**
Get the lowest singular value of specific sparse matrix

In [5]:
using SparseArrays: SparseMatrixCSC, sparse, nnz
sp = kron(SparseMatrixCSC(cnot), sparse(I, 100, 100))

# install `KrylovKit`
using KrylovKit: eigsolve
# ev = <get lowest eigenvalue of sp>

@testset "sparse" begin
    @test sp |> nnz == 100*4    # here `x |> f` is same as calling f(x).
    @test vals[1] ≈ minimum(eigen(sp |> Matrix).values)
end

sparse: Error During Test at In[5]:10
  Test threw exception
  Expression: vals[1] ≈ minimum((eigen(sp |> Matrix)).values)
  UndefVarError: vals not defined
  Stacktrace:
   [1] macro expansion at ./In[5]:10 [inlined]
   [2] macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Test/src/Test.jl:1083 [inlined]
   [3] top-level scope at ./In[5]:9
Test Summary: | Pass  Error  Total
sparse        |    1      1      2


TestSetException: Some tests did not pass: 1 passed, 0 failed, 1 errored, 0 broken.

# Tensor contraction

[TensorOperations](https://github.com/Jutho/TensorOperations.jl) is a high performance package for tensor contractions.

### **Used in**
* Tensor Networks (TRG, DMRG,...)


### **Challenge!**
Try to calculate the following contraction
$C_{lj} = A_{i,j,k}B_{i,k,l}$

In [6]:
A = randn(6, 10, 5)
B = randn(6, 5, 7)

# install TensorOperations
using TensorOperations
# @tensor C[a, b] := <specify contraction>

@test C |> size == (7, 10)

Error During Test at In[6]:8
  Test threw exception
  Expression: C |> size == (7, 10)
  UndefVarError: C not defined
  Stacktrace:
   [1] top-level scope at In[6]:6
   [2] eval at ./boot.jl:319 [inlined]
   [3] softscope_include_string(::Module, ::String, ::String) at /home/leo/.julia/packages/SoftGlobalScope/ujmiK/src/SoftGlobalScope.jl:206
   [4] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/leo/.julia/packages/IJulia/k5o7j/src/execute_request.jl:165
   [5] #invokelatest#1 at ./essentials.jl:686 [inlined]
   [6] invokelatest at ./essentials.jl:685 [inlined]
   [7] eventloop(::ZMQ.Socket) at /home/leo/.julia/packages/IJulia/k5o7j/src/eventloop.jl:8
   [8] (::getfield(IJulia, Symbol("##12#15")))() at ./task.jl:259


Test.FallbackTestSetException: There was an error during testing

# Save and load data

[DelimitedFiles](https://docs.julialang.org/en/v1/stdlib/DelimitedFiles/index.html) is the `txt` format save and load standard module.

[JLD2 and FileIO](https://github.com/simonster/JLD2.jl) uses HDF5 format to save and load, suited for large data file.


### **Challenge!**
Read out the saved data.

In [7]:
using DelimitedFiles

a = randn(Float64, 3,3)
writedlm("data/_test.dat", a)
# b = <read data from file `data/_test.dat`>

# FileIO
# install FileIO and JLD2
using FileIO, JLD2
jldopen("data/_example.jld2", "w") do f
    f["A"] = a
end

# b_jld2 = <read data from file `data/_example.jld2`>
@testset "file reading" begin
    @test b ≈ a
    @test b_jld2 ≈ a
end

file reading: Error During Test at In[7]:16
  Test threw exception
  Expression: b ≈ a
  DimensionMismatch("dimensions must match")
  Stacktrace:
   [1] promote_shape at ./indices.jl:129 [inlined]
   [2] promote_shape at ./indices.jl:125 [inlined]
   [3] promote_shape(::Array{Complex{Float64},1}, ::Array{Float64,2}) at ./indices.jl:120
   [4] - at ./arraymath.jl:38 [inlined]
   [5] #isapprox#21(::Int64, ::Float64, ::Bool, ::typeof(LinearAlgebra.norm), ::Function, ::Array{Complex{Float64},1}, ::Array{Float64,2}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/LinearAlgebra/src/generic.jl:1339
   [6] isapprox(::Array{Complex{Float64},1}, ::Array{Float64,2}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/LinearAlgebra/src/generic.jl:1339
   [7] eval_test(::Expr, ::Expr, ::LineNumberNode) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Test/src/Test.jl:229
   [8] macro expansion at ./In[7]:16 [inlined]
   [9] macro

TestSetException: Some tests did not pass: 0 passed, 0 failed, 2 errored, 0 broken.

# `################################################################`
# Now, you can embrace the world of Notebooks!
# To embrace the programmer's world, we need an **ADVANCED SESSION** ;D
# `################################################################`

# Dispatch Functions

We have [multiple dispatch](https://docs.julialang.org/en/v1/manual/functions/) + [type tree](https://docs.julialang.org/en/v1/manual/types/) instead of classes in object oriented design.

In [30]:
"""This is a utility of showing subtype tree."""
function subtypetree(t, level=1, indent=4)
   level == 1 && println(t)
   for s in subtypes(t)
     println(join(fill(" ", level * indent)) * string(s))
     subtypetree(s, level+1, indent)
   end
end

subtypetree

In [31]:
subtypetree(Number)

Number
    Complex
    Real
        AbstractFloat
            BigFloat
            Float16
            Float32
            Float64
        AbstractIrrational
            Irrational
        Integer
            Bool
            Signed
                BigInt
                Int128
                Int16
                Int32
                Int64
                Int8
            TensorOperations.One
            TensorOperations.Zero
            Unsigned
                UInt128
                UInt16
                UInt32
                UInt64
                UInt8
        Rational


In [41]:
dump(Array)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Core.TypeofBottom Union{}
    ub: Any
  body: UnionAll
    var: TypeVar
      name: Symbol N
      lb: Core.TypeofBottom Union{}
      ub: Any
    body: Array{T,N} <: DenseArray{T,N}


### **Challendge!**
Fix following tests

In [42]:
@testset "types" begin
    @test 1.0 isa Float64
    @test 1.0 isa Real
    @test typeof(1.0) == Float64
    
    # type relation
    @test Int64 <: Int
    @test Int64 === Int
    @test Int64 <: Integer
    @test Int64 <: Union{Int64, Complex}
    @test Array{ComplexF64, 3} <: Array{ComplexF64}
    @test Array{ComplexF64, 3} <: Array{Complex, 3}
    @test supertype(Integer) == Real
    @test Signed in subtypes(Integer)
    
    # type promotion
    @test eltype(promote(1.0, 2im)) == Complex
    @test promote_type(Float32, Float64) == Float64
    @test promote_type(Int64, Real, Float64) == Float64
    
    # element types
    @test eltype([1, 2, 3.0]) == Vector{Float64}
    @test eltype(Int[1, 2, 3.0]) == Int64
    
end

types: Test Failed at In[42]:12
  Expression: Array{ComplexF64, 3} <: Array{Complex, 3}
Stacktrace:
 [1] macro expansion at ./In[42]:12 [inlined]
 [2] macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Test/src/Test.jl:1083 [inlined]
 [3] top-level scope at ./In[42]:2
types: Test Failed at In[42]:17
  Expression: eltype(promote(1.0, 2im)) == Complex
   Evaluated: Complex{Float64} == Complex
Stacktrace:
 [1] macro expansion at ./In[42]:17 [inlined]
 [2] macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Test/src/Test.jl:1083 [inlined]
 [3] top-level scope at ./In[42]:2
types: Test Failed at In[42]:19
  Expression: promote_type(Int64, Real, Float64) == Float64
   Evaluated: Real == Float64
Stacktrace:
 [1] macro expansion at ./In[42]:19 [inlined]
 [2] macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Test/src/Test.jl:1083 [inlined]
 [3] top-level scope at ./In[42]:2
types: T

TestSetException: Some tests did not pass: 12 passed, 4 failed, 0 errored, 0 broken.

In [8]:
#=
    define function f here
=#

@testset "functions" begin
    @test f(3) == 9
    @test f.([1,2,3,4,5]) == [1,4,9,16,25]  # broadcast
    @test f("bili") == "bilibili"
    @test f.(["bili", "dili"]) == ["bilibili", "dilidili"]

    x = [1, 2, 3]
    y = [1im, 2+3im, 3]
    @test f(x) == [1,2,3,1,2,3]  # repeat the array
    @test f(y) == [1im,2+3im,3,-1im,2-3im,3]  # repeat, but with conjugate
    y = copy(x)
    @test (f!(y); y) == [1,2,3,1,2,3]  # inplace version of f
end

functions: Error During Test at In[8]:6
  Test threw exception
  Expression: f(3) == 9
  UndefVarError: f not defined
  Stacktrace:
   [1] macro expansion at ./In[8]:6 [inlined]
   [2] macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Test/src/Test.jl:1083 [inlined]
   [3] top-level scope at ./In[8]:6
functions: Error During Test at In[8]:7
  Test threw exception
  Expression: f.([1, 2, 3, 4, 5]) == [1, 4, 9, 16, 25]
  UndefVarError: f not defined
  Stacktrace:
   [1] macro expansion at ./In[8]:7 [inlined]
   [2] macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Test/src/Test.jl:1083 [inlined]
   [3] top-level scope at ./In[8]:6
functions: Error During Test at In[8]:8
  Test threw exception
  Expression: f("bili") == "bilibili"
  UndefVarError: f not defined
  Stacktrace:
   [1] macro expansion at ./In[8]:8 [inlined]
   [2] macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1

TestSetException: Some tests did not pass: 0 passed, 0 failed, 7 errored, 0 broken.

# Type Stability

[Type stability](https://docs.julialang.org/en/v1/manual/performance-tips/index.html#Avoid-changing-the-type-of-a-variable-1) is the hardest and most important part of writing high performance Julia programs.

### **Challenge!**
Fix the allocation below to increase the performance of calculating Fibonacci.

In [9]:
# fix Fibonacci to improve performance
function fib(n)
    a = 0
    b = 0b1
    for i = 1:n-1
        c = a + b
        a = b
        b = c
    end
    b
end
display(@benchmark fib(30))
display(@code_warntype fib(30))

BenchmarkTools.Trial: 
  memory estimate:  256 bytes
  allocs estimate:  16
  --------------
  minimum time:     908.486 ns (0.00% GC)
  median time:      958.135 ns (0.00% GC)
  mean time:        1.487 μs (21.40% GC)
  maximum time:     2.283 ms (99.95% GC)
  --------------
  samples:          10000
  evals/sample:     37

Body::Union{Int64, UInt8}
│╻     -5  1 ── %1  = (Base.sub_int)(n, 1)::Int64
││╻╷╷╷  Type   │    %2  = (Base.sle_int)(1, %1)::Bool
│││╻     unitrange_last   │          (Base.sub_int)(%1, 1)
││││     │    %4  = (Base.ifelse)(%2, %1, 0)::Int64
││╻╷╷   isempty   │    %5  = (Base.slt_int)(%4, 1)::Bool
││       └───       goto #3 if not %5
││       2 ──       goto #4
││       3 ──       goto #4
│        4 ┄─ %9  = φ (#2 => true, #3 => false)::Bool
│        │    %10 = φ (#3 => 1)::Int64
│        │    %11 = (Base.not_int)(%9)::Bool
│        └───       goto #15 if not %11
│        5 ┄─ %13 = φ (#4 => 0, #14 => %14)::Union{Int64, UInt8}
│        │    %14 = φ (#4 => 0x01, #14 => %36)::Union{Int64, UInt8}
│        │    %15 = φ (#4 => %10, #14 => %42)::Int64
│     6  │    %16 = (isa)(%13, UInt8)::Bool
│        │    %17 = (isa)(%14, Int64)::Bool
│        │    %18 = (and_int)(%16, %17)::Bool
│        └───       goto #7 if not %18
│        6 ── %20 = π (%13, UInt8

nothing

)
│        │    %21 = π (%14, Int64)
││╻╷╷╷  rem   │    %22 = (Core.zext_int)(Core.Int64, %20)::Int64
││╻     +   │    %23 = (Base.add_int)(%22, %21)::Int64
│        └───       goto #10
│        7 ── %25 = (isa)(%13, Int64)::Bool
│        │    %26 = (isa)(%14, UInt8)::Bool
│        │    %27 = (and_int)(%25, %26)::Bool
│        └───       goto #9 if not %27
│        8 ── %29 = π (%13, Int64)
│        │    %30 = π (%14, UInt8)
││╻╷╷╷  rem   │    %31 = (Core.zext_int)(Core.Int64, %30)::Int64
││╻     +   │    %32 = (Base.add_int)(%29, %31)::Int64
│        └───       goto #10
│        9 ── %34 = (%13 + %14)::Union{Int64, UInt8}
│        └───       goto #10
│        10 ┄ %36 = φ (#6 => %23, #8 => %32, #9 => %34)::Union{Int64, UInt8}
│╻╷    iterate8  │    %37 = (%15 === %4)::Bool
││       └───       goto #12 if not %37
││       11 ─       goto #13
││╻     +   12 ─ %40 = (Base.add_int)(%15, 1)::Int64
│╻     iterate   └───       goto #13
│        13 ┄ %42 = φ (#12 => %40)::Int64
│        │    %

In [10]:
@testset "fibonacci" begin
    @test fib(30) == 832040
    @test (@allocated fib(30))  == 0
end

fibonacci: Test Failed at In[10]:3
  Expression: #= In[10]:3 =# @allocated(fib(30)) == 0
   Evaluated: 256 == 0
Stacktrace:
 [1] macro expansion at ./In[10]:3 [inlined]
 [2] macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Test/src/Test.jl:1083 [inlined]
 [3] top-level scope at ./In[10]:2
Test Summary: | Pass  Fail  Total
fibonacci     |    1     1      2


TestSetException: Some tests did not pass: 1 passed, 1 failed, 0 errored, 0 broken.

# Final Boss

Give data file `data/example.txt`, with each row a tuple of $i, j, w_{ij}$.
The problems is to find the ground state configuration of the classical Ising hamiltonian $H = \sum\limits_{i,j} w_{ij}\sigma_i \sigma_j$

This is the code chanlledge of 2016 UCAS summer school.

### Simmulated Annealling
Fix the code of simulated annealing

1. correctify the code
2. improve the performance

In [75]:
# please open `simulated_annealing.ipynb` to continue